### PETR6397 Final Project: Oil Production Forecasting using Machine Learning

In [7]:
# Import libraries

import pandas as pd
import numpy as np
import zipfile
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.pyplot as plt
import plotly.graph_objects as go

from pmdarima.arima import auto_arima
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings("ignore")

# Load the two data files from zip folder into one dataframe

# read the first CSV file into a DataFrame
df1 = pd.read_csv('Dataset/Producing Entity Monthly Production 1.CSV')

# read the second CSV file into another DataFrame
df2 = pd.read_csv('Dataset/Producing Entity Monthly Production 2.CSV')

# concatenate the two DataFrames vertically
df = pd.concat([df1, df2], axis=0)

# print the resulting DataFrame
df.head(10)

,Entity ID,API/UWI,API/UWI List,Monthly Production Date,Monthly Oil,Monthly Gas,Monthly Water,Well Count,Days,Daily Avg Oil,Daily Avg Gas,Daily Avg Water,Reservoir,Well/Lease Name,Well Number,Operator Company Name,Production Type,Production Status,Entity Type,Producing Month Number
0,130854587,4.213538e+13,4213538451,2019-12-01,1.0,139.0,7,1,NaN,0.03,4.48,0.23,CLEAR FORK,SCIMITAR,1,OCCIDENTAL PETROLEUM,OIL,ACTIVE,COM,1
1,130854587,4.213538e+13,4213538451,2020-01-01,22.0,270.0,1205,1,NaN,0.71,8.71,38.87,CLEAR FORK,SCIMITAR,1,OCCIDENTAL PETROLEUM,OIL,ACTIVE,COM,2
2,130854587,4.213538e+13,4213538451,2020-02-01,15.0,130.0,846,1,NaN,0.52,4.48,29.17,CLEAR FORK,SCIMITAR,1,OCCIDENTAL PETROLEUM,OIL,ACTIVE,COM,3
3,130854587,4.213538e+13,4213538451,2020-03-01,14.0,308.0,788,1,NaN,0.45,9.94,25.42,CLEAR FORK,SCIMITAR,1,OCCIDENTAL PETROLEUM,OIL,ACTIVE,COM,4
4,130854587,4.213538e+13,4213538451,2020-04-01,13.0,268.0,773,1,NaN,0.43,8.93,25.77,CLEAR FORK,SCIMITAR,1,OCCIDENTAL PETROLEUM,OIL,ACTIVE,COM,5
5,130854587,4.213538e+13,4213538451,2020-05-01,9.0,31.0,546,1,NaN,0.29,1.00,17.61,CLEAR FORK,SCIMITAR,1,OCCIDENTAL PETROLEUM,OIL,ACTIVE,COM,6
6,130854587,4.213538e+13,4213538451,2020-06-01,6.0,0.0,395,1,NaN,0.20,0.00,13.17,CLEAR FORK,SCIMITAR,1,OCCIDENTAL PETROLEUM,OIL,ACTIVE,COM,7
7,130854587,4.213538e+13,4213538451,2020-07-01,6.0,244.0,435,1,NaN,0.19,7.87,14.03,CLEAR FORK,SCIMITAR,1,OCCIDENTAL PETROLEUM,OIL,ACTIVE,COM,8
8,130854587,4.213538e+13,4213538451,2020-08-01,12.0,262.0,832,1,NaN,0.39,8.45,26.84,CLEAR FORK,SCIMITAR,1,OCCIDENTAL PETROLEUM,OIL,ACTIVE,COM,9
9,130854587,4.213538e+13,4213538451,2020-09-01,10.0,188.0,698,1,NaN,0.33,6.27,23.27,CLEAR FORK,SCIMITAR,1,OCCIDENTAL PETROLEUM,OIL,ACTIVE,COM,10


In [13]:
# check the shape of the DataFrame

print("Shape of the DataFrame:", df.shape)

Shape of the DataFrame: (773296, 20)


In [14]:
# check the data types of the columns

print("\nData types of columns:\n", df.dtypes)


Data types of columns:
 Entity ID                    int64
API/UWI                    float64
API/UWI List                object
Monthly Production Date     object
Monthly Oil                float64
Monthly Gas                float64
Monthly Water                int64
Well Count                   int64
Days                       float64
Daily Avg Oil              float64
Daily Avg Gas              float64
Daily Avg Water            float64
Reservoir                   object
Well/Lease Name             object
Well Number                 object
Operator Company Name       object
Production Type             object
Production Status           object
Entity Type                 object
Producing Month Number       int64
dtype: object


In [15]:
# check for missing values

print("\nNumber of missing values:\n", df.isna().sum())


Number of missing values:
 Entity ID                       0
API/UWI                         0
API/UWI List                    0
Monthly Production Date         0
Monthly Oil                   380
Monthly Gas                   356
Monthly Water                   0
Well Count                      0
Days                       746292
Daily Avg Oil                 380
Daily Avg Gas                 356
Daily Avg Water                 0
Reservoir                    2684
Well/Lease Name                 0
Well Number                  6866
Operator Company Name           0
Production Type                 0
Production Status               0
Entity Type                     0
Producing Month Number          0
dtype: int64


In [18]:
# summary statistics for numerical columns

df.describe()

,Entity ID,API/UWI,Monthly Oil,Monthly Gas,Monthly Water,Well Count,Days,Daily Avg Oil,Daily Avg Gas,Daily Avg Water,Producing Month Number
count,7.732960e+05,7.732960e+05,772916.000000,772940.000000,7.732960e+05,773296.000000,27004.0,772916.000000,772940.000000,773296.000000,773296.000000
mean,1.262313e+08,4.213525e+13,536.239141,546.452097,7.396827e+03,1.020856,0.0,17.616824,17.949356,243.004030,257.563919
std,1.323235e+07,1.675612e+08,1102.827386,2890.248581,1.860082e+04,0.271829,0.0,36.216032,94.893978,611.029891,194.551139
min,3.043260e+05,4.213500e+13,0.000000,0.000000,0.000000e+00,1.000000,0.0,0.000000,0.000000,0.000000,1.000000
25%,1.273338e+08,4.213508e+13,91.000000,15.000000,6.180000e+02,1.000000,0.0,2.970000,0.480000,20.300000,95.000000
50%,1.275302e+08,4.213530e+13,229.000000,68.000000,2.978000e+03,1.000000,0.0,7.540000,2.230000,97.810000,214.000000
75%,1.277265e+08,4.213538e+13,560.000000,283.000000,7.416000e+03,1.000000,0.0,18.390000,9.320000,243.275000,389.000000
max,1.310139e+08,4.213590e+13,35382.000000,190132.000000,1.411641e+06,9.000000,0.0,1141.350000,6133.290000,45536.810000,997.000000
